<a href="https://colab.research.google.com/github/MurilloMMartins/piquete_pitico/blob/main/piquete_pitico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Piquete Pitico
Davi Fagundes

Eduardo Fernandes

Murillo Martins

Vítor Fróis

### Music recommendation from an image


# Install and Configure Colab

In [ ]:
%pip install cohere
%pip install annoy
%pip install altair
%pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 27.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552448 sha256=27b12cb8e0174cce0c9063bc9c79c743a91f06a290ef7d905166c6043bd7906e
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━

In [ ]:
import json
kj = {"username":"vafrois","key":""}
json.dump(kj, open("kaggle.json", "w"))

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import cohere
import pandas as pd
import sklearn
import kaggle
import numpy as np
import umap.umap_ as umap
import altair as alt
from annoy import AnnoyIndex

Get Cohere API

In [ ]:
co = cohere.Client('')

# Similarity Metrics

### Cosine Similarity

In [ ]:
def cos_sim(a, b):
    return (a @ b.T) / (np.linalg.norm(a)*np.linalg.norm(b))

### Nearest Neighbors Search

In [ ]:
class SearchIndex:
  def create_search_index(self, embeds):
    # Create the search index, pass the size of embedding
    search_index = AnnoyIndex(np.array(embeds).shape[1], 'angular')
    # Add all the vectors to the search index
    for i in range(len(embeds)):
      search_index.add_item(i, embeds[i])
    search_index.build(10) # 10 trees
    search_index.save('test.ann')
    self.search_index = search_index

  def get_index(self, item_text, n_ranks=10):
    similar_item_ids = self.search_index.get_nns_by_vector(item_text,n_ranks,
                                                  include_distances=True)
    return similar_item_ids


# Visualize Dataset

## Spotify Dataset

In [ ]:
!kaggle datasets download imuhammad/audio-features-and-lyrics-of-spotify-songs
!unzip /content/audio-features-and-lyrics-of-spotify-songs.zip

 41% 5.00M/12.3M [00:00<00:00, 43.0MB/s]
100% 12.3M/12.3M [00:00<00:00, 86.8MB/s]
Archive:  /content/audio-features-and-lyrics-of-spotify-songs.zip
  inflating: spotify_songs.csv       


In [ ]:
spodf = pd.read_csv("spotify_songs.csv")
spodf.head()

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,...,-10.068,1,0.0236,0.27900,0.01170,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,...,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,...,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en


In [ ]:
# Lendo os dados
spodf = pd.read_csv('spotify_songs.csv', sep = ',')
# filtrando
spodf.drop_duplicates(subset='track_name', inplace=True)
spodf = spodf[spodf['language'] == 'en']
spodf = spodf[['track_name', 'track_artist', 'lyrics','playlist_genre', 'language', 'track_popularity', 'danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', ]]
spodf = spodf[spodf['track_popularity'] > 30]
spotify_df = spodf[spodf['lyrics'].apply(lambda x: x.isascii())].head(5000)
spotify_df['lyrics'] = spotify_df['lyrics'].str.replace("'", "")
spotify_df.head()

,track_name,track_artist,lyrics,playlist_genre,language,track_popularity,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence
3,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really cant stay Baby its cold outside Ive g...,r&b,en,41,0.425,0.378,0.0341,0.6890,0.000000,0.0664,0.405
5,Soldier,James TW,"Hold your breath, dont look down, keep trying ...",r&b,en,70,0.496,0.639,0.0550,0.2800,0.000000,0.0975,0.305
6,Satisfy You,Diddy,All I want is somebody whos gonna love me for ...,r&b,en,52,0.764,0.594,0.1850,0.5910,0.000000,0.1450,0.695
7,Tender Lover,Babyface,Feels good Everybody Tender lover Tender love ...,r&b,en,36,0.743,0.860,0.0445,0.2260,0.000422,0.0513,0.687
8,Hide Away (feat. Envy Monroe),Blasterjaxx,"Dont run away, its getting colder Our hearts u...",edm,en,42,0.573,0.746,0.0421,0.0249,0.000000,0.3610,0.134


In [ ]:
#spotify_list = ["  ".join([f"{col}: {val}" for col, val in row.items()]) for _, row in spotify_df.iterrows()]
spotify_list = []
for _, row in spotify_df.iterrows():
  aux = "  ".join([f"{col}: {val}" for col, val in row[:4].items()])
  if row['danceability'] > 0.5:
    aux += '  danceable'

  if row['energy'] > 0.5:
    aux += '  energetic'

  if row['speechiness'] > 0.33:
    aux += '  speechable'

  if row['acousticness'] > 0.5:
    aux += '  acoustic'

  if row['instrumentalness'] > 0.25:
    aux += '  instrumental'

  if row['liveness'] > 0.8:
    aux += '  lively'

  aux += '  feeling: '
  if row['valence'] > 0.75:
    aux += 'positive'
  elif row['valence'] < 0.25:
    aux += 'negative'
  else:
    aux += 'neutral'

  spotify_list.append(aux)

spotify_list[:1]

["track_name: Baby It's Cold Outside (feat. Christina Aguilera)  track_artist: CeeLo Green  lyrics: I really cant stay Baby its cold outside Ive got to go away Baby its cold out there This evening has been Been hoping that youd drop in So very nice Ill hold your hands, theyre just like ice My mother will start to worry Beautiful, whats your hurry? My father will be pacing the floor Listen to that fireplace roar So really Id better scurry Beautiful, please dont hurry Well maybe just a half a drink more Why dont you put some records on while I pour The neighbors might think Baby, its bad out there Say, whats in this drink? No cabs to be had out there I wish I knew how Your eyes are like starlight To break the spell Ill take your hat, your hair looks swell I ought to say no, no, no, sir Mind if I move in closer? At least Im gonna say that I tried Whats the sense in hurting my pride? I really cant stay Baby dont hold out Baby its cold outside I simply must go See that its cold outside The 

In [ ]:
len(spotify_list)

5000

## Flickr Dataset

In [ ]:
! kaggle datasets download hsankesara/flickr-image-dataset/ -f flickr30k_images/results.csv
! unzip results.csv.zip

  0% 0.00/3.59M [00:00<?, ?B/s]
100% 3.59M/3.59M [00:00<00:00, 79.0MB/s]
Archive:  results.csv.zip
  inflating: results.csv             


In [ ]:
raw_flickr_df = pd.read_csv("results.csv", sep="|")
raw_flickr_df.head()

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .


In [ ]:
# filtering by selected images
selected_images = [
    '1001545525.jpg',
    '1001633352.jpg',
    '100444898.jpg',
    '1011572216.jpg',
    '101362133.jpg',
    '1027149103.jpg',
    '104136873.jpg',
    '104763189.jpg',
    '106027774.jpg',
    '1100214449.jpg',
    '1107507919.jpg'
    ]

# image_desc_df = image_desc_df.loc[image_desc_df["image_name"].isin(selected_images)]
image_desc_df = raw_flickr_df.head(500)
aux = image_desc_df.loc[image_desc_df[' comment_number'] == " 0"]
desc_df = aux[["image_name", " comment"]]
desc_df.columns = ["image_name", "comment"]
desc_df

,image_name,comment
0,1000092795.jpg,Two young guys with shaggy hair look at their...
5,10002456.jpg,Several men in hard hats are operating a gian...
10,1000268201.jpg,A child in a pink dress is climbing up a set ...
15,1000344755.jpg,Someone in a blue shirt and hat is standing o...
20,1000366164.jpg,"Two men , one in a gray shirt , one in a blac..."
...,...,...
475,102851549.jpg,A little boy looks at the camera while a woma...
480,102860573.jpg,A construction site on a street with three me...
485,10287332.jpg,Two men sitting on the roof of a house while ...
490,1028982826.jpg,"Two men , one younger guy with a yellow shirt..."


In [ ]:
list_desc = list(desc_df['comment'])
list_desc[:5]

[' Two young guys with shaggy hair look at their hands while hanging out in the yard .',
 ' Several men in hard hats are operating a giant pulley system .',
 ' A child in a pink dress is climbing up a set of stairs in an entry way .',
 ' Someone in a blue shirt and hat is standing on stair and leaning against a window .',
 ' Two men , one in a gray shirt , one in a black shirt , standing near a stove .']

# Cohere  

### Summarize

In [ ]:
text = """
Come wit' it now
The microphone explodes, shattering the molds
Either drop the hits like De La O or get the fuck off the commode
Wit' the sure shot, sure to make the bodies drop
Drop and don't copy yo, don't call this a co-op
Terror rains drenchin', quenchin' the thirst of the power dons
That five sided fist-a-gon
The rotten sore on the face of mother earth gets bigger
The triggers cold empty ya purse
Rally 'round the family with a pocket full of shells
They rally 'round the family with a pocket full of shells
"""

In [ ]:
response = co.summarize(
    text=text,
    model='command',
    length='short',
    extractiveness='high'
)

summary = response.summary

In [ ]:
summary

"The microphone explodes, shattering the molds. Either drop the hits or get off the commode. Terror rains, quenching the thirst of the power dons. They rally 'round the family with a pocket full of shells."

# Embed and Inference

Create Music Embeddings

In [ ]:
spotify_embeddings = co.embed(texts=spotify_list,
                  model="embed-english-v2.0").embeddings

In [ ]:
len(spotify_embeddings)

5000

In [ ]:
spotify_df['embeddings'] = spotify_embeddings
spotify_df

,track_name,track_artist,lyrics,playlist_genre,language,track_popularity,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,embeddings
3,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really cant stay Baby its cold outside Ive g...,r&b,en,41,0.425,0.378,0.0341,0.6890,0.000000,0.0664,0.405,"[-0.28100586, 0.66796875, -0.76464844, 3.02148..."
5,Soldier,James TW,"Hold your breath, dont look down, keep trying ...",r&b,en,70,0.496,0.639,0.0550,0.2800,0.000000,0.0975,0.305,"[1.1474609, 0.2130127, 0.019760132, 1.6064453,..."
6,Satisfy You,Diddy,All I want is somebody whos gonna love me for ...,r&b,en,52,0.764,0.594,0.1850,0.5910,0.000000,0.1450,0.695,"[0.3503418, 0.9199219, -0.4519043, 1.296875, 2..."
7,Tender Lover,Babyface,Feels good Everybody Tender lover Tender love ...,r&b,en,36,0.743,0.860,0.0445,0.2260,0.000422,0.0513,0.687,"[-0.014045715, 2.1757812, -0.013343811, 1.8066..."
8,Hide Away (feat. Envy Monroe),Blasterjaxx,"Dont run away, its getting colder Our hearts u...",edm,en,42,0.573,0.746,0.0421,0.0249,0.000000,0.3610,0.134,"[0.015106201, 0.10028076, 1.1835938, 0.8642578..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12488,The Git Up,Blanco Brown,Right now I just need you to get real loose Ge...,pop,en,52,0.847,0.678,0.1090,0.0669,0.000000,0.2740,0.811,"[1.7597656, 0.9213867, -1.8457031, 0.97265625,..."
12490,No Stress - Sofi Tukker Remix,Laurent Wolf,(Jeremy Hills/Eric Rima/Laurent Wolf) I dont w...,edm,en,40,0.584,0.928,0.0320,0.0261,0.057000,0.1610,0.065,"[-1.1611328, 1.3095703, -1.3330078, 0.89746094..."
12494,Hold Me While You Wait - Steve Void Remix,Lewis Capaldi,"Im waitin up, savin all my precious time Losin...",pop,en,63,0.675,0.760,0.0434,0.1040,0.000000,0.3300,0.468,"[1.1533203, 1.4287109, 0.6196289, 0.7636719, -..."
12495,Birthday,JP Cooper,"Oh, we had something Had something bubbling It...",r&b,en,55,0.644,0.581,0.0418,0.0314,0.000000,0.1240,0.266,"[-0.3425293, 2.484375, -0.20715332, 0.62158203..."


In [ ]:
spotify_df.to_csv("embeddings.csv")

KeyboardInterrupt: ignored

In [ ]:
spotify_si = SearchIndex()
spotify_si.create_search_index(spotify_embeddings)

Get embedding for image description

In [ ]:
# desc_to_compare = list_desc[0]
desc_to_compare = "starry night over the city by van gogh  feeling: happy angry"

In [ ]:
desc_embeddings = co.embed(texts=[desc_to_compare],
                  model="embed-english-v2.0").embeddings

In [ ]:
similar_item_ids = spotify_si.get_index(desc_embeddings[0], 10) #n_ranks = 10 by default

In [ ]:
similar_item_ids

([2196, 1601, 841, 4117, 2703, 3645, 2514, 1292, 781, 4786],
 [0.9573585987091064,
  0.9632493853569031,
  0.9638025760650635,
  0.9685813188552856,
  0.9727442264556885,
  0.9744170308113098,
  0.9758864641189575,
  0.9788529872894287,
  0.9799972176551819,
  0.9931591749191284])

Predict most similar musics through Nearest Neighbor Search

In [ ]:
results = pd.DataFrame(data={'texts': spotify_df.iloc[similar_item_ids[0]]['track_name'],
                             'artist': spotify_df.iloc[similar_item_ids[0]]['track_artist'],
                             'distance': similar_item_ids[1]})
print(f"Question:'{desc_to_compare}'\nNearest neighbors:")
results

Question:'starry night over the city by van gogh  feeling: happy angry'
Nearest neighbors:


,texts,artist,distance
5070,But Not Tonight,Depeche Mode,0.957359
3551,Paranoid,Black Sabbath,0.963249
1841,My Time,Jordan Rakei,0.963803
10034,Beauty and Madness - 1993,Fra Lippo Lippi,0.968581
6364,Dear Friends - Remastered 2011,Queen,0.972744
8735,Lights,Journey,0.974417
5844,Sad Song (feat. TINI),Alesso,0.975886
2852,Anthem Of The Lonely,Nine Lashes,0.978853
1699,Van Gogh's Crows,Shahmen,0.979997
11940,Sunflower (feat. Steve Lacy),Vampire Weekend,0.993159


Visualize Spotify Embeddings

In [ ]:
# UMAP reduces the dimensions from 1024 to 2 dimensions that we can plot
reducer = umap.UMAP(n_neighbors=20)
umap_embeds = reducer.fit_transform(spotify_embeddings)
# Prepare the data to plot and interactive visualization
# using Altair
df_explore = pd.DataFrame(data={'text': list(spotify_df['track_name'])})
df_explore['x'] = umap_embeds[:,0]
df_explore['y'] = umap_embeds[:,1]

# Plot
chart = alt.Chart(df_explore).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    tooltip=['text']
).properties(
    width=700,
    height=400
)
chart.interactive()

In [ ]:
umap_embeds

# Tests with Segmentation

Tried to segmentation image into 4 new images, get descriptions for each one and embedding for the concat.

It didnt go as expected

In [ ]:
img1 = 'there is a man sitting at a table with a laptop computer'
img2 = 'smiling man with two fingers up in front of a laptop'
img3 = 'there are two men sitting at a table with laptops'
img4 = 'there is a person sitting at a table with a laptop computer'
emotion = 'neutral'
img_total = img1+img2+img3+img4+emotion

In [ ]:
desc_embeddings = co.embed(texts=[img_total],
                  model="embed-english-v2.0").embeddings

In [ ]:
response = co.chat(
	"Please recommend a music to post in a Instragram Stories photo with the following description: there is a man sitting at a table with a laptop computer neutral",
	model="command",
	temperature=0.9
)

answer = response.text


In [ ]:
answer